In [2]:
import pymysql #Connector library for mysql
import re
import nltk
from nltk.tokenize import *
from nltk.corpus import stopwords
from nltk.stem import *
from nltk import *

# list of keywords
keywords=['food','atmosphere','place','order','time','service','restaurant','price','eat','taste','menu','experience','location','area','customer','clean']

# Stem the review
def stem(word):
    wnl = WordNetLemmatizer()
    temp_text=wnl.lemmatize(word,'v')
    return wnl.lemmatize(temp_text,'n')

# Tokenize
def tokenize(review):
    tok = TreebankWordTokenizer().tokenize(review)
    return tok

# Remove stopwords
def remove_stop(review):
    stopset = set(stopwords.words('english'))
    result_text= [w for w in review if not w in stopset]
    return result_text

#Check if all upper
def all_upper(h):
    if(len([letter for letter in h if re.compile('[A-Z]').findall(letter)])==len(h)):
        return True

    #Set up a connection with the server
try:
    conn = pymysql.connect(host = 'localhost', port = 3306, user = 'root', passwd = 'root', db='YELP_DATA_PROJECT')
except:
    print("I am unable to connect to the database")
#And a cursor object that will serve as a virtual 'cursor'
curr = conn.cursor()
extract_usa_review=('Select* from usa_reviews limit 700;')
curr.execute(extract_usa_review)
results=curr.fetchall()
reviews_text=""
for row in results:
    reviews_id=row[0]
    user_id=row[1]
    business_id=row[2]
    date=row[3]
    text=row[4]
    votes=row[5]
    stars=row[6]
    type_data=row[7]
    
    if (len(text)<=150):
        print("----------The Review (less than 150)------------")
        print(text)
    else:
        print("---------Long review------")
        print(text)
        tokenized_review=tokenize(text)
        a=[]
        summary=[]
        for word in tokenized_review:
            word_fix=word.replace(".","",100)
            word_fix=word_fix.replace("?","",100)
            word_fix=word_fix.replace("!","",100)
            word_fix=word_fix.replace(",","",100)
            word_lower=word_fix.lower()
            stemmed_word=stem(word_lower)
            # Get the sentences containing keywords
            if(stemmed_word in keywords and stemmed_word not in a):
                pattern=r"([^.?!]*?"+str(word)+"\\b[^.?!]*[?!\.]+)" # \\b = boundary of the word:Can be space, punctuation...
                match=re.findall(pattern,str(text.replace("/n","",100)))
                a.append(str(word_lower))
                for item in match:
                    summary.append(str(item))
            # Get the snetences contianing all caps words
            elif (len(word_fix)>1 and all_upper(word_fix)):
                pattern=r"([^.?!]*?"+str(word)+"\\b[^.?!]*[?!\.]+)" # \\b = boundary of the word:Can be space, punctuation...
                match=re.findall(pattern,str(text.replace("/n","",100)))
                a.append(str(word_lower))
                for item in match:
                    summary.append(str(item))
        
        # Get the sentences containing at least 2 exclamation points !!   
        match=re.findall(r"([^.?!]*?[^.?!]*!!+)",str(text))
        if match: # True findall returned something
             for item in match:
                    summary.append(str(item))
                    
        print("----The summary----")
        summary = map(lambda s: s.strip(), summary) #Remove the /n
        summary=set(summary)
        for item in summary:
            print(str(item))
    print("")

---------Long review------
My girlfriend and I ate here in 2012. We both took violently I'll within 12 hours. We alerted the manager in person, via phone call from both of us and in writing via email. Their response was one of denial, lack of empathy and a complete disregard for customer service. We notified them not to finger point but out of public health concern and Casey the manager was put off by this. She stated do what you need to do to make yourself feel better. I caution you to eat here, they may make you sick. They don't care about their customers,  suggest try another steak place.
----The summary----
I caution you to eat here, they may make you sick.
My girlfriend and I ate here in 2012.
They don't care about their customers,  suggest try another steak place.
Their response was one of denial, lack of empathy and a complete disregard for customer service.

----------The Review (less than 150)------------
The food is pretty good. I got the greek salad with grilled chicken, whi